In [1]:
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd

In [2]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from tokenizers import Tokenizer

/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# df_row = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_oneclaim_1000_198_200.csv', engine='python')
# df_claim_cpc = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc.csv', engine='python')
df_claim_cpc_1000 = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc_all_1000.csv', encoding='ISO-8859-1')

In [4]:
df = df_claim_cpc_1000

In [5]:
df = df.rename(columns={"text_x": "text", "section_id_y": "section_id", "subsection_id_y": "subsection_id","group_id_y": "group_id", "subgroup_id_y": "subgroup_id"})

In [6]:
df.head()

,patent_id,text,section_id,subsection_id,group_id,subgroup_id
0,10606736,A computer-implemented method for creation of ...,G,G06,G06F,G06F11/3612
1,10606738,"A method, comprising: receiving results from a...",H,H04,H04L,H04L2209/56
2,10606739,"A device, comprising: a memory; and one or mor...",G,G06,G06F,G06F11/3684
3,10606740,A system comprising: a processor comprising a ...,"G,G,G","G06,G06,G06","G06F,G06F,G06F","G06F12/0207,G06F2212/455,G06F9/4881"
4,10606741,A process performed by a computing device for ...,G,G06,G06F,G06F3/0673


In [7]:
total_patents = df.count()
total_patents

patent_id        1000
text             1000
section_id        998
subsection_id     998
group_id          998
subgroup_id       998
dtype: int64

In [8]:
claims = list(df.text)
patent_id = list(df.patent_id)

In [9]:
# claims

In [10]:
def get_top_n_similar_patents_df(new_claim, claims):
    # embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
    embedder = SentenceTransformer('stsb_augsbert_SS_roberta-base-2021-01-06_22-14-54')
#     embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
    query_embeddings = embedder.encode([new_claim])

#     query_embeddings = tokenizer([new_claim], padding=True, truncation=True, max_length=128, return_tensors='pt')

    # list of patent claims
    claim_embeddings = embedder.encode(claims)

    # get top 100 patent claims based on cosine similarity
    closest_n = 3
    distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    # save similar patents info
    top_claim_ids = []
    top_claims = []
    top_similarity_scores = []


    print('New_claim: ' + new_claim + '\n')

    # Find the closest 100 patent claims for each query new_claim based on cosine similarity
    for idx, distance in results[0:closest_n]:
        top_claim_ids.append(patent_id[idx])
        top_claims.append(claims[idx])
        top_similarity_scores.append(round((1-distance), 4))
        print('Patent ID: ' + str(patent_id[idx]))
        print('PubMed Claim: ' + claims[idx])
        print('Similarity Score: ' + "%.4f" % (1-distance))
        print('\n')
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
        'claims': top_claims,
    })
    
    return top_100_similar_patents_df

In [11]:
new_claim = ['The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.']

In [12]:
# !pip install tokenizer
# from tokenizer import tokenize

In [13]:
new_claim

['The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.']

In [14]:
# search for similar patent claim

if os.path.exists('top_100_similar_patents_df.csv'):
    get_top_n_similar_patents_df = pd.read_csv('top_100_similar_patents_df.csv')

else:
    get_top_n_similar_patents_df = get_top_n_similar_patents_df(new_claim[0], claims)

New_claim: The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.

Patent ID: 10606970
PubMed Claim: A computer-implemented method for statistical static timing analysis of an integrated circuit, the method comprising: specifying, by a user, a criteria for paths and tests for the integrated circuit; performing, by a processor, a statistical static timing analysis for each of the paths of the integrated 

In [15]:
get_top_n_similar_patents_df.head()

,top_claim_ids,cosine_similarity,claims
0,10606970,0.7243,A computer-implemented method for statistical ...
1,10606874,0.6766,A method implemented in a computer infrastruct...
2,10606858,0.6512,A method for presenting content based on a gen...


In [16]:
result = pd.concat([get_top_n_similar_patents_df, df], axis=1, join='inner')

In [17]:
# result.loc[result['top_claim_ids'], result["section_id"]] = set(result["section_id"])
# result.loc[result['section_id'].apply(lambda x: set(result.section_id.iloc[:1]))] 

In [17]:
result.to_csv(r'/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc_all_result.csv', index = False)

In [3]:
# result.section_id.iloc[:1]
df_claim_cpc_result = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc_all_result.csv', encoding='ISO-8859-1')

In [20]:
# df_claim_cpc_result.head()
df_claim_cpc_result = result
df_claim_cpc_result.head()
# result.head()

,top_claim_ids,cosine_similarity,claims,patent_id,text,section_id,subsection_id,group_id,subgroup_id
0,10606970,0.7243,A computer-implemented method for statistical ...,10606736,A computer-implemented method for creation of ...,G,G06,G06F,G06F11/3612
1,10606874,0.6766,A method implemented in a computer infrastruct...,10606738,"A method, comprising: receiving results from a...",H,H04,H04L,H04L2209/56
2,10606858,0.6512,A method for presenting content based on a gen...,10606739,"A device, comprising: a memory; and one or mor...",G,G06,G06F,G06F11/3684


In [4]:
# df_claim_cpc_result.at[3,'section_id'] = set(df_claim_cpc_result.at[3,'section_id'])

In [5]:
# df_claim_cpc_result.at[3,'section_id']
# df_claim_cpc_result.section_id = df_claim_cpc_result.section_id.duplicated()
# df_claim_cpc_result['section_id'].str.split(', ').apply(set).str.join(', ')
df_claim_cpc_result['section_id'] = df_claim_cpc_result['section_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_claim_cpc_result['subsection_id'] = df_claim_cpc_result['subsection_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_claim_cpc_result['group_id'] = df_claim_cpc_result['group_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_claim_cpc_result['subgroup_id'] = df_claim_cpc_result['subgroup_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [25]:
# result['section_id'] = set(result['section_id'])

In [40]:
# print(result.count())

In [21]:
k_similar_patents = df_claim_cpc_result.nlargest(10, ['cosine_similarity'])

In [22]:
k_similar_patents

,top_claim_ids,cosine_similarity,claims,patent_id,text,section_id,subsection_id,group_id,subgroup_id
0,10606970,0.7243,A computer-implemented method for statistical ...,10606736,A computer-implemented method for creation of ...,G,G06,G06F,G06F11/3612
1,10606874,0.6766,A method implemented in a computer infrastruct...,10606738,"A method, comprising: receiving results from a...",H,H04,H04L,H04L2209/56
2,10606858,0.6512,A method for presenting content based on a gen...,10606739,"A device, comprising: a memory; and one or mor...",G,G06,G06F,G06F11/3684


In [23]:
final_result_mod = k_similar_patents.mode()
# final_result_max = k_similar_patents.max()
print('Prediction for section_id of New Patent is: ', final_result_mod.iloc[0]['section_id'])
print('Prediction for section_id of New Patent is: ', k_similar_patents.iloc[0:10]['section_id'])
print('Prediction for subsection_id of New Patent is: ', k_similar_patents.iloc[0:10]['subsection_id'])
print('Prediction for group_id of New Patent is: ', k_similar_patents.iloc[0:10]['group_id'])
print('Prediction for subgroup_id of New Patent is: ', k_similar_patents.iloc[0:10]['subgroup_id'])

Prediction for section_id of New Patent is:  G
Prediction for section_id of New Patent is:  0    G
1    H
2    G
Name: section_id, dtype: object
Prediction for subsection_id of New Patent is:  0    G06
1    H04
2    G06
Name: subsection_id, dtype: object
Prediction for group_id of New Patent is:  0    G06F
1    H04L
2    G06F
Name: group_id, dtype: object
Prediction for subgroup_id of New Patent is:  0    G06F11/3612
1    H04L2209/56
2    G06F11/3684
Name: subgroup_id, dtype: object
